In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import pathlib
import time
import datetime
import numpy as np
import scipy.io as sio
from matplotlib import pyplot as plt
from IPython import display

In [ ]:
exp_name   = "Experiments_combined"
test_path  = "despeckle/" + "Experiments_IV" + "/test_volumes/Volume_01" # Test dataset directory 
train_path = "despeckle/" + exp_name + "/test" #Train dataset directory
log_path   = "despeckle/" + exp_name + "/logs" # directory to save log files
ckpt_path  = "despeckle/" + exp_name + "/ckpt_32_32_16bit_Discriminator_126_126_latest" # directory to save checkpoints
save_dir   = "/data/PersonalFolders/bhaskara/despeckle/" + "Experiments_IV" + "/result_volumes/Volume_01_3D_combined/" # directory to save results
isExist = os.path.exists(save_dir)
if not isExist:
    os.mkdir(save_dir)

In [ ]:
#image_path = train_path + "/[p.SHARP][s.Eye2a][10-09-2019_13-14-42]_500.mat"
#try: 
#    sample_image = sio.loadmat(image_path)['bscanStack']
#except:
#    sample_image = sio.loadmat(image_path)['image_stack']
##sample_image = tf.io.read_file(image_path)
##sample_image = tf.io.decode_mat(sample_image)
#print(sample_image.shape)
#plt.figure()
#im = plt.imshow(sample_image[:,:,0 ],cmap='gray')
#plt.colorbar(im)
#print(type(sample_image))
#print(sample_image.dtype)

In [ ]:
def load_test(image_path):
  # Read and decode an image file to a uint8 tensor
  def read_mat(image_path):
    # Input mat files are 10*log10(tomogram intensity)
    try:
        image = sio.loadmat(image_path.numpy())['bscanStack']
    except:
        image = sio.loadmat(image_path.numpy())['image_stack']
        
    #noiseFloordB = np.round(np.mean(image[1:20,200:250,:]))
#     noiseFloordB = noiseFloordB+2.50
    noiseFloordB = 50;
    maximumdB = 250;
    # Compute the noise Floor
#     try:
#        ind = np.unravel_index(np.argmax(image, axis=None), image.shape)
#        maximumdB = np.round(np.mean(image[ind(0)-5:ind(0)+5, ind(1)-5:ind(1)+5, ind(2)-2:ind(2)+2]))   
#     except:
#        maximumdB = np.max(image[:,:,8])
                             
    image = (image-noiseFloordB)/(maximumdB-noiseFloordB)
    image = image*(2**16-1)
    image = np.clip(image, 0, 2**16-1)
    data = image.astype(np.uint16) 
    return data
  image = tf.py_function(func=read_mat, inp=[image_path], Tout=tf.uint16)
  #image = tf.convert_to_tensor(image, dtype=tf.uint8)
  image = tf.image.convert_image_dtype(image, dtype=tf.uint16, saturate=False)
  maxImageDimension = tf.math.reduce_max(tf.shape(image))
  image = tf.image.resize_with_crop_or_pad(image, maxImageDimension, maxImageDimension)
    # Convert both images to float32 tensors
  input_image_stack = tf.cast(image, tf.float32)
  #real_image = tf.cast(real_image, tf.float32)
  return input_image_stack

In [ ]:
#inp = load_test(image_path)
## Casting to int for mat
#print(inp.shape)
##print(re.shape)

#plt.figure(figsize=(15, 15))
#for i in range(9):
#  plt.subplot(3, 3, i+1)
#  im = plt.imshow(inp[:,:, 2*i+1], vmin= 0, vmax= 2**16-1, cmap='gray')
#  plt.axis('off')
#  plt.colorbar(im)
#plt.show()

In [ ]:
def resize(input_image_stack, height, width):
  input_image_stack = tf.image.resize_with_crop_or_pad(input_image_stack, height, width)
  #real_image = tf.image.resize(real_image, [height, width],method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
  return input_image_stack

In [ ]:
# Normalizing the images to [-1, 1]
def normalize(input_image_stack):
  input_image_stack = (input_image_stack/(0.5*65535))-1
  #input_image_stack = (input_image_stack / (0.5*tf.math.reduce_max(input_image_stack))) - 1
  #real_image = (real_image / 127.5) - 1
  return input_image_stack

In [ ]:
# The batch size of 1 produced better results for the U-Net in the original pix2pix experiment
BATCH_SIZE = 1
# Each image is 1024 x 1024 in size
IMG_WIDTH = 1024
IMG_HEIGHT = 1024

In [ ]:
def load_image_test(image_file):
  input_image_stack = load_test(image_file)
  input_image_stack = resize(input_image_stack, IMG_HEIGHT, IMG_WIDTH)
  input_image_stack = normalize(input_image_stack)
  return input_image_stack[:,:,:17], input_image_stack[:,:,17:], image_file

In [ ]:
#inp, re, filename = load_image_test(image_path)
#print(inp.shape)
#print(re.shape)

#im= plt.imshow(inp[:,:,8], cmap='gray')
#plt.colorbar(im)
#plt.axis('off')
#plt.show()

#im= plt.imshow(re, cmap='gray')
#plt.colorbar(im)
#plt.axis('off')
#plt.show()

#print(np.max(inp[:,:,8]))
#print(np.max(inp))
#print(np.max(re))

In [ ]:
try:
  test_dataset = tf.data.Dataset.list_files(test_path + "/*.mat")
except tf.errors.InvalidArgumentError:
  test_dataset = tf.data.Dataset.list_files(test_path + "/*.mat")
print(test_dataset.cardinality().numpy())
test_dataset = test_dataset.map(load_image_test)
test_dataset = test_dataset.batch(BATCH_SIZE)

In [ ]:
OUTPUT_CHANNELS = 1

In [ ]:
def downsample(filters, size, apply_batchnorm=True):
  initializer = tf.random_normal_initializer(0., 0.02)

  result = tf.keras.Sequential()
  result.add(
      tf.keras.layers.Conv2D(filters, size, strides=2, padding='same',
                             kernel_initializer=initializer, use_bias=True))

  if apply_batchnorm:
    result.add(tf.keras.layers.BatchNormalization())

  result.add(tf.keras.layers.LeakyReLU())

  return result

In [ ]:
def upsample(filters, size, apply_dropout=False):
  initializer = tf.random_normal_initializer(0., 0.02)

  result = tf.keras.Sequential()
  result.add(
    tf.keras.layers.Conv2DTranspose(filters, size, strides = 2,
                                    padding = 'same',
                                    kernel_initializer = initializer,
                                    use_bias = True))

  result.add(tf.keras.layers.BatchNormalization())

  if apply_dropout:
      result.add(tf.keras.layers.Dropout(0.5))

  result.add(tf.keras.layers.ReLU())

  return result

In [ ]:
def Generator():
  inputs = tf.keras.layers.Input(shape=[1024, 1024, 17])

  down_stack = [
    downsample(256, 4, apply_batchnorm=False),  # (batch_size, 512, 512, 64)
    downsample(512, 4),  # (batch_size, 256, 256, 128)
    downsample(1024, 4),  # (batch_size, 128, 128, 256)
    downsample(2048, 4),  # (batch_size, 64, 64, 512)
    downsample(2048, 4),  # (batch_size, 32, 32, 512)
    #downsample(2048, 4),  # (batch_size, 16, 16, 512)
    #downsample(2048, 4),  # (batch_size, 8, 8, 512)
    #downsample(2048, 4),  # (batch_size, 4, 4, 512)
    #downsample(2048, 4),  # (batch_size, 2, 2, 512)
    #downsample(512, 4),  # (batch_size, 1, 1, 512)
  ]

  up_stack = [
    #upsample(2048, 4, apply_dropout=True),  # (batch_size, 2, 2, 1024)
    #upsample(2048, 4, apply_dropout=True),  # (batch_size, 4, 4, 1024)
    #upsample(2048, 4),  # (batch_size, 8, 8, 1024)
    upsample(2048, 4),  # (batch_size, 16, 16, 1024)
    upsample(1024, 4),  # (batch_size, 32, 32, 1024)
    #upsample(512, 4),  # (batch_size, 64, 64, 1024)
    upsample(512, 4),  # (batch_size, 128, 128, 512)
    upsample(256, 4),  # (batch_size, 256, 256, 256)
    #upsample(64, 4),  # (batch_size, 512, 512, 128)
  ]

  initializer = tf.random_normal_initializer(0., 0.02)
  last = tf.keras.layers.Conv2DTranspose(OUTPUT_CHANNELS, 4,
                                         strides = 2,
                                         padding = 'same',
                                         kernel_initializer = initializer,
                                         activation = 'tanh')  # (batch_size, 1024, 1024, 1)

  x = inputs

  # Downsampling through the model
  skips = []
  for down in down_stack:
    x = down(x)
    skips.append(x)

  skips = reversed(skips[:-1])

  # Upsampling and establishing the skip connections
  for up, skip in zip(up_stack, skips):
    x = up(x)
    x = tf.keras.layers.Concatenate()([x, skip])

  x = last(x)

  return tf.keras.Model(inputs=inputs, outputs=x)

In [ ]:
generator = Generator()
tf.keras.utils.plot_model(generator, show_shapes=True, dpi=64)

In [ ]:
# def Discriminator():
#   initializer = tf.random_normal_initializer(0., 0.02)
#   inp = tf.keras.layers.Input(shape=[1024, 1024, 17], name='input_image')
#   tar = tf.keras.layers.Input(shape=[1024, 1024, 1], name='target_image')
#   x = tf.keras.layers.concatenate([inp, tar])  # (batch_size, 1024, 1024, channels*2)
#   down1 = downsample(256, 4, False)(x)  # (batch_size, 512, 512, 64)
#   down2 = downsample(512, 4)(down1)  # (batch_size, 256, 256, 128)
#   down3 = downsample(1024, 4)(down2)  # (batch_size, 128, 128, 256)
#   #down4 = downsample(512, 4)(down3)  # (batch_size, 64, 64, 512)
#   #down5 = downsample(1024, 4)(down4)  # (batch_size, 32, 32, 1024)
#   zero_pad1 = tf.keras.layers.ZeroPadding2D()(down3)  # (batch_size, 130, 130, 256)
#   conv = tf.keras.layers.Conv2D(2048, 4, strides=1,
#                                 kernel_initializer=initializer,
#                                 use_bias=False)(zero_pad1)  # (batch_size, 127, 127, 512)
#   batchnorm1 = tf.keras.layers.BatchNormalization()(conv)
#   leaky_relu = tf.keras.layers.LeakyReLU()(batchnorm1)
#   zero_pad2 = tf.keras.layers.ZeroPadding2D()(leaky_relu)  # (batch_size, 129, 129, 512)
#   last = tf.keras.layers.Conv2D(1, 4, strides=1,
#                                 kernel_initializer=initializer)(zero_pad2)  # (batch_size, 126, 126, 1)
#   return tf.keras.Model(inputs=[inp, tar], outputs=last)

In [ ]:
def Discriminator():
  initializer = tf.random_normal_initializer(0., 0.02)
  inp = tf.keras.layers.Input(shape=[1024, 1024, 17], name='input_image')
  tar = tf.keras.layers.Input(shape=[1024, 1024, 1], name='target_image')
  x = tf.keras.layers.concatenate([inp, tar])  # (batch_size, 1024, 1024, channels*2)
  down1 = downsample(512, 4, False)(x)  # (batch_size, 512, 512, 64)
  down2 = downsample(1024, 4)(down1)  # (batch_size, 256, 256, 128)
  down3 = downsample(1024, 4)(down2)  # (batch_size, 128, 128, 256)
  #down4 = downsample(512, 4)(down3)  # (batch_size, 64, 64, 512)
  #down5 = downsample(1024, 4)(down4)  # (batch_size, 32, 32, 1024)
  zero_pad1 = tf.keras.layers.ZeroPadding2D()(down3)  # (batch_size, 130, 130, 256)
  conv = tf.keras.layers.Conv2D(2048, 4, strides=1,
                                kernel_initializer=initializer,
                                use_bias=False)(zero_pad1)  # (batch_size, 127, 127, 512)
  batchnorm1 = tf.keras.layers.BatchNormalization()(conv)
  leaky_relu = tf.keras.layers.LeakyReLU()(batchnorm1)
  zero_pad2 = tf.keras.layers.ZeroPadding2D()(leaky_relu)  # (batch_size, 129, 129, 512)
  last = tf.keras.layers.Conv2D(1, 4, strides=1,
                                kernel_initializer=initializer)(zero_pad2)  # (batch_size, 126, 126, 1)
  return tf.keras.Model(inputs=[inp, tar], outputs=last)

In [ ]:
discriminator = Discriminator()
tf.keras.utils.plot_model(discriminator, show_shapes=True, dpi=64)

In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

In [ ]:
checkpoint_dir = "/data/PersonalFolders/bhaskara/" + ckpt_path
checkpoint_prefix = os.path.join(checkpoint_dir, "")

In [ ]:
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

In [ ]:
def generate_images(model, test_input, tar, filename):
  start =  time.time()
  prediction = model(test_input, training=True)
  timeExec = round(time.time()-start,3)  
  print(f'Time taken: {time.time()-start:.2f} sec\n')
  panel = tf.concat([tf.squeeze(test_input[0][:,:,8]), tf.squeeze(tar[0]), tf.squeeze(prediction[0])],1)
  panel = (panel+1)*32767.5
  #panel = panel.astype(np.uint16)
  #panel_cast = tf.cast((panel+1)*127.5, dtype='uint8')
  #panel_cast = tf.cast((panel+1)*32767.5, dtype='uint16')
  mdict = {"outputStack": panel.numpy()}
  #panel = tf.expand_dims(panel,2)
  #encoded = tf.io.encode_png(tf.cast((panel+1)*127.5, dtype='uint8'))
  #encoded = tf.io.encode_png(tf.cast((tf.squeeze(prediction, axis = 0)+1)*127.5, dtype='uint8'))
  #outputfile = tf.io.write_file(filename, encoded)
  sio.savemat(filename, mdict)
  plt.figure(figsize=(15, 15))
  display_list = [test_input[0][:,:,8], tar[0], prediction[0]]
  mae = tf.keras.losses.MeanAbsoluteError()
  print(f'L1_error:{mae(tar[0], prediction[0]).numpy()}')
  title = ['Input Image', 'Ground Truth', 'Predicted Image']
  for i in range(3):
    plt.subplot(1, 3, i+1)
    plt.title(title[i])
    # Getting the pixel values in the [0, 1] range to plot.
    im = plt.imshow(tf.squeeze((display_list[i]+1)*32767.5), cmap='gray')
    #plt.colorbar(im)
    plt.axis('off')   
  plt.show()
  return timeExec

In [ ]:
# 
# print(log_path)
# log_dir = "/data/PersonalFolders/bhaskara/" + log_path
# %reload_ext tensorboard
# %tensorboard --logdir "/data/PersonalFolders/bhaskara/despeckle"

In [ ]:
# Restoring the latest checkpoint in checkpoint_dir
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [ ]:
# Run the trained model on a few examples from the test set
all = test_dataset.cardinality().numpy()
timeExeclist = []
for inp, tar, img in test_dataset.take(all):
  f_name = str(img.numpy())
  #f_name = save_dir + f_name
  f_name = f_name.split('/')[-1]
  f_name = f_name[:-2]
  f_name = save_dir + f_name 
  print(f_name)
  timeExec = generate_images(generator, inp, tar, f_name)
  timeExeclist.append(timeExec)

In [ ]:
print(np.mean(timeExeclist[1:]))